![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+Forecasting&file=Vertex+AI+AutoML+Forecasting+-+Tabular+Workflows.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20Forecasting/Vertex%20AI%20AutoML%20Forecasting%20-%20Tabular%20Workflows.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520Forecasting%2FVertex%2520AI%2520AutoML%2520Forecasting%2520-%2520Tabular%2520Workflows.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20Forecasting/Vertex%20AI%20AutoML%20Forecasting%20-%20Tabular%20Workflows.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20Forecasting/Vertex%20AI%20AutoML%20Forecasting%20-%20Tabular%20Workflows.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Vertex AI AutoML Forecasting - Tabular Workflows

Overview coming soon!

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [22]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google_cloud_pipeline_components', 'google-cloud-pipeline-components')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

## API Enablement

In [5]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [6]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [10]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [31]:
REGION = 'us-central1'
EXPERIMENT = 'tft-workflows'
SERIES = 'applied-forecasting'

# GCS Bucket name
GCS_BUCKET= PROJECT_ID

# BQ Table Parameters
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-','_')
BQ_TABLE = 'forecasting-data_prepped'

viz_limit = 12

Packages

In [32]:
import os
import uuid
from google.cloud import aiplatform
from google.cloud import bigquery
from google_cloud_pipeline_components.preview.automl import forecasting

Clients

In [25]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


parameters:

In [35]:
DIR = f"temp/{SERIES}-{EXPERIMENT}"
UUID = str(uuid.uuid4())
UUID

'46ff34f1-8a8f-40da-9194-7b9ee2c72f50'

In [15]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [16]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## Temporal Fusion Transformer With Tabular Workflows 

- https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.11.0/api/preview/automl/forecasting.html#preview.automl.forecasting.get_temporal_fusion_transformer_forecasting_pipeline_and_parameters

In [38]:
# CUSTOMIZE
TARGET_COLUMN = 'num_trips'
TIME_COLUMN = 'starttime'
SERIES_COLUMN = 'start_station_name'
SPLIT_COLUMN = 'splits'
#COVARIATE_COLUMNS = ['avg_tripduration', 'pct_subscriber', 'ratio_gender', 'capacity'] # could be empty
COVARIATE_COLUMNS_ATTRIBUTES = []
COVARIATE_COLUMNS_KNOWN = ['capacity']
COVARIATE_COLUMNS_UNKNOWN = ['avg_tripduration', 'pct_subscriber', 'ratio_gender']

# CUSTOMIZE
#FORECAST_GRANULARITY = 'DAY' # the data preparation included preparing the data at this level
FORECAST_HORIZON_LENGTH = 14
#FORECAST_TEST_LENGTH = 14 # the data preparation included setting this value for splits = TEST
#FORECAST_VALIDATE_LENGTH = 14 # the data preparation included setting this value for splits = VALIDATE

# PARAMETERS
FEATURES = [TIME_COLUMN, TARGET_COLUMN] + COVARIATE_COLUMNS_ATTRIBUTES + COVARIATE_COLUMNS_KNOWN + COVARIATE_COLUMNS_UNKNOWN
ROOT_DIR = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root/{UUID}'
OPTIMIZATION_OBJECTIVE = 'minimize-mae'
BUDGET_MILLI_HOURS = 6*1000
BQ_SOURCE = f'bq://{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}'

Review table header:

In [26]:
bq.query(f'SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` LIMIT 5').to_dataframe()

,start_station_name,starttime,num_trips,avg_tripduration,pct_subscriber,ratio_gender,capacity,splits
0,Central Park S & 6 Ave,2013-09-01,256,2349.875000,0.222656,0.190698,73,TRAIN
1,Central Park S & 6 Ave,2015-06-21,256,2148.695312,0.203125,0.201878,73,TRAIN
2,Central Park S & 6 Ave,2016-06-07,256,1375.144531,0.621094,0.651613,73,TRAIN
3,Central Park S & 6 Ave,2016-02-28,256,2330.507812,0.265625,0.242718,73,TRAIN
4,Central Park S & 6 Ave,2015-09-11,256,2085.941406,0.300781,0.319588,73,TRAIN


Retrieve Key Dates for splits:

In [28]:
query = f"""
    WITH
        SPLIT AS (
            SELECT splits, min({TIME_COLUMN}) as mindate, max({TIME_COLUMN}) as maxdate
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
            GROUP BY {SPLIT_COLUMN}
        ),
        TRAIN AS (
            SELECT mindate as start_date
            FROM SPLIT
            WHERE {SPLIT_COLUMN} ='TRAIN'
        ),
        VAL AS (
            SELECT mindate as val_start
            FROM SPLIT
            WHERE {SPLIT_COLUMN} = 'VALIDATE'
        ),
        TEST AS (
            SELECT mindate as test_start, maxdate as end_date
            FROM SPLIT
            WHERE {SPLIT_COLUMN} = 'TEST'
        )
    SELECT * EXCEPT(pos) FROM
    (SELECT *, ROW_NUMBER() OVER() pos FROM TRAIN)
    JOIN (SELECT *, ROW_NUMBER() OVER() pos FROM VAL)
    USING (pos)
    JOIN (SELECT *, ROW_NUMBER() OVER() pos FROM TEST)
    USING (pos)
"""
keyDates = bq.query(query).to_dataframe()
keyDates

,start_date,val_start,test_start,end_date
0,2013-07-01,2016-09-03,2016-09-17,2016-09-30


In [29]:
try:
    model = aiplatform.Model(
        model_name = f'model-{SERIES}-{EXPERIMENT}'
    )
    parent_model = model.resource_name
    print('Found previous version for this model.')
except Exception:
    parent_mmodel = ''
    print('This is a first run for this model.')

This is a first run for this model.


In [37]:
dataset = aiplatform.TimeSeriesDataset.list(filter = f'display_name={SERIES}')[0]
if not dataset:
    print('Creating dataset.')
    dataset = aiplatform.TimeSeriesDataset.create(
        display_name = f'{SERIES}', 
        bq_source = f'bq://{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}'
    )
else:
    print('Found dataset from prior run.')

Found dataset from prior run.


In [41]:
(template_path, parameter_values) = forecasting.get_temporal_fusion_transformer_forecasting_pipeline_and_parameters(
    project = PROJECT_ID,
    location = REGION,
    root_dir = ROOT_DIR,
    target_column = TARGET_COLUMN,
    optimization_objective = OPTIMIZATION_OBJECTIVE,
    transformations = dict(auto = FEATURES),
    train_budget_milli_node_hours = BUDGET_MILLI_HOURS,
    #data_source_csv_filenames = data_source_csv_filenames,
    data_source_bigquery_table_path = BQ_SOURCE,
    weight_column = None,
    predefined_split_key = SPLIT_COLUMN,
    #training_fraction = training_fraction,
    #validation_fraction = validation_fraction,
    #test_fraction = test_fraction,
    # Please note that TFT model will ONLY ensemble the model from
    # the top one trial, so `num_selected_trials` can not be set for TFT model.
    # num_selected_trials=num_selected_trials,
    time_column = TIME_COLUMN,
    time_series_identifier_columns = [SERIES_COLUMN],
    time_series_attribute_columns = COVARIATE_COLUMNS_ATTRIBUTES,
    available_at_forecast_columns = [TIME_COLUMN] + COVARIATE_COLUMNS_KNOWN,
    unavailable_at_forecast_columns = [TARGET_COLUMN] + COVARIATE_COLUMNS_UNKNOWN,
    forecast_horizon = FORECAST_HORIZON_LENGTH,
    context_window = FORECAST_HORIZON_LENGTH,
    dataflow_subnetwork = None,
    dataflow_use_public_ips = True,
    run_evaluation = True,
    evaluated_examples_bigquery_path = f'bq://{BQ_PROJECT}.{BQ_DATASET}',
    dataflow_service_account = SERVICE_ACCOUNT,
    # Quantile prediction is NOT supported by TFT.
    holiday_regions = ['GLOBAL', 'NA', 'EMEA', 'JAPAC', 'LAC'],
    group_total_weight = 1.0,
    temporal_total_weight = 2.0,
    group_temporal_total_weight = 1.0,
)

In [42]:
job = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}-{UUID}",
    location = REGION,  # launches the pipeline job in the specified region
    template_path = template_path,
    job_id = f"{SERIES}-{EXPERIMENT}-{UUID}",
    pipeline_root = ROOT_DIR,
    parameter_values = parameter_values,
    enable_caching = False,
)

job.run(service_account = SERVICE_ACCOUNT)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/applied-forecasting-tft-workflows-46ff34f1-8a8f-40da-9194-7b9ee2c72f50
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/applied-forecasting-tft-workflows-46ff34f1-8a8f-40da-9194-7b9ee2c72f50')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/applied-forecasting-tft-workflows-46ff34f1-8a8f-40da-9194-7b9ee2c72f50?project=1026793852137
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/applied-forecasting-tft-workflows-46ff34f1-8a8f-40da-9194-7b9ee2c72f50 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/applied-forecasting-tft-workflows-46ff34f1-8a8f-40da-9194-7b9ee2c72f50 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projec

In [49]:
for task in job.task_details:
    print(task.task_name)

applied-forecasting-tft-workflows-46ff34f1-8a8f-40da-9194-7b9ee2c72f50
feature-attribution-2
get-or-create-model-description-2
condition-2
automl-tabular-finalizer
model-batch-predict-2
string-not-empty
automl-forecasting-ensemble-2
get-predictions-column-2
model-upload-2
feature-transform-engine
set-optional-inputs
model-evaluation-import-2
calculate-training-parameters-2
condition-5
table-to-uri-2
model-batch-explanation-2
split-materialized-data
finalize-eval-quantile-parameters-2
get-prediction-image-uri-2
condition-4
exit-handler-1
model-evaluation-forecasting-2
training-configurator-and-validator
automl-forecasting-stage-1-tuner


In [50]:
for task in job.task_details:
    if task.task_name == 'model-upload-2':
        model_task = task

In [52]:
model_task.outputs['model']

artifacts {
  name: "projects/1026793852137/locations/us-central1/metadataStores/default/artifacts/5751720186074823162"
  display_name: "model"
  uri: "https://us-central1-aiplatform.googleapis.com/v1/projects/1026793852137/locations/us-central1/models/7091833506480783360@1"
  etag: "1711091955879"
  create_time {
    seconds: 1711091617
    nanos: 921000000
  }
  update_time {
    seconds: 1711091955
    nanos: 879000000
  }
  state: LIVE
  schema_title: "google.VertexModel"
  schema_version: "0.0.1"
  metadata {
    fields {
      key: "resourceName"
      value {
        string_value: "projects/1026793852137/locations/us-central1/models/7091833506480783360@1"
      }
    }
  }
}

In [53]:
forecast_model = aiplatform.Model(model_task.outputs['model'].artifacts[0].metadata['resourceName'])

---
### Get Forecasted Values for Future Horizon
Use a batch prediction job with the resulting forecasting model to get predicted forecast for the future horizon. 

The requirement for getting batch predictions from a Vertex AI forecasting model are covered [here](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting/get-predictions).

#### Prepare Input Table in BigQuery
For Vetex AI forecasting batch prediction jobs we need a table (BigQuery or CSV) that contains a row per date in the forecast horizon and history for atleast the length of the context window. The following BigQuery query constructs this input table from the source data while also filling in missing dates in the context window with the last known observation.

In [58]:
context_window = FORECAST_HORIZON_LENGTH

query_a = ""
query_b = ""
for v in COVARIATE_COLUMNS_KNOWN + COVARIATE_COLUMNS_UNKNOWN + COVARIATE_COLUMNS_ATTRIBUTES:
    query_a += f""",
            LAST_VALUE({v} IGNORE NULLS) OVER (PARTITION BY {SERIES_COLUMN} ORDER BY {TIME_COLUMN} ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as {v}"""
    if v not in COVARIATE_COLUMNS_ATTRIBUTES:
        query_b += f""",
        CASE WHEN {TIME_COLUMN} > (SELECT MAX({TIME_COLUMN}) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`) THEN NULL ELSE {v} END AS {v}"""
    else:
        query_b += f""",
        {v}"""

query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{EXPERIMENT}_horizon_input` AS
WITH
    DATELIST AS (
        SELECT *
        FROM (SELECT DISTINCT {SERIES_COLUMN} FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`) A
        CROSS JOIN (SELECT * 
                    FROM UNNEST(GENERATE_DATE_ARRAY(
                                    DATE_SUB((SELECT MAX({TIME_COLUMN}) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`), INTERVAL {context_window-1} DAY),
                                    DATE_ADD((SELECT MAX({TIME_COLUMN}) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`), INTERVAL {FORECAST_HORIZON_LENGTH} DAY),
                                    INTERVAL 1 DAY
                                )
                            ) AS {TIME_COLUMN}
                    ) B
    ),
    ADDTARGET AS (
        SELECT *
        FROM DATELIST
        LEFT OUTER JOIN (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
        USING ({SERIES_COLUMN}, {TIME_COLUMN})
        ORDER BY {SERIES_COLUMN}, {TIME_COLUMN}
    ),
    LOCF AS (
        SELECT {SERIES_COLUMN}, {TIME_COLUMN},
        LAST_VALUE({TARGET_COLUMN} IGNORE NULLS) OVER (PARTITION BY {SERIES_COLUMN} ORDER BY {TIME_COLUMN} ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as {TARGET_COLUMN}
        {query_a}
        FROM ADDTARGET
    )
SELECT {SERIES_COLUMN}, {TIME_COLUMN},
    CASE
        WHEN {TIME_COLUMN} > (SELECT MAX({TIME_COLUMN}) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`) THEN NULL
        ELSE {TARGET_COLUMN}
    END AS {TARGET_COLUMN}
    {query_b}
FROM LOCF
ORDER BY {SERIES_COLUMN}, {TIME_COLUMN}
"""
job = bq.query(query = query)
job.result()

In [65]:
bq.query(f"SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{EXPERIMENT}_horizon_input` WHERE start_station_name = 'Central Park West & W 68 St' ORDER BY starttime").to_dataframe()

,start_station_name,starttime,num_trips,capacity,avg_tripduration,pct_subscriber,ratio_gender
0,Central Park West & W 68 St,2016-09-17,251,43,1610.055777,0.557769,0.619355
1,Central Park West & W 68 St,2016-09-18,169,43,1270.952663,0.538462,0.707071
2,Central Park West & W 68 St,2016-09-19,70,43,902.771429,0.914286,1.916667
3,Central Park West & W 68 St,2016-09-20,189,43,1155.597884,0.756614,1.223529
4,Central Park West & W 68 St,2016-09-21,131,43,1066.061069,0.786260,1.183333
5,Central Park West & W 68 St,2016-09-22,182,43,1036.373626,0.708791,1.044944
6,Central Park West & W 68 St,2016-09-23,205,43,1379.707317,0.702439,1.029703
7,Central Park West & W 68 St,2016-09-24,211,43,1303.521327,0.568720,0.635659
8,Central Park West & W 68 St,2016-09-25,210,43,1303.890476,0.585714,0.707317
9,Central Park West & W 68 St,2016-09-26,122,43,1028.680328,0.598361,0.876923


#### Batch Prediction Job

Request the batch prediction directly using the `batch_prediction()` method for the model - [SDK Reference](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_batch_predict).

In [60]:
batchjob = forecast_model.batch_predict(
    job_display_name = f'{SERIES}_{EXPERIMENT}_predictions',
    bigquery_source = f"bq://{BQ_PROJECT}.{BQ_DATASET}.{EXPERIMENT}_horizon_input",
    instances_format = 'bigquery',
    bigquery_destination_prefix = f"bq://{BQ_PROJECT}.{BQ_DATASET}",
    predictions_format = 'bigquery',
    sync = True
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/7155302746474676224
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/1026793852137/locations/us-central1/batchPredictionJobs/7155302746474676224')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/7155302746474676224?project=1026793852137
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/7155302746474676224 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/7155302746474676224 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/7155302746474676224 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredicti

In [61]:
batchjob.output_info.bigquery_output_table

'predictions_2024_03_22T04_23_27_121Z_659'

In [66]:
preds = bq.query(f"SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{batchjob.output_info.bigquery_output_table}`  WHERE start_station_name = 'Central Park West & W 68 St' ORDER BY starttime, predicted_on_starttime").to_dataframe()
preds

,avg_tripduration,capacity,num_trips,pct_subscriber,predicted_num_trips,predicted_on_starttime,ratio_gender,start_station_name,starttime
0,NaN,__MISSING__,None,NaN,"{'value': 207.7970428466797, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-01
1,NaN,__MISSING__,None,NaN,"{'value': 226.3101043701172, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-02
2,NaN,__MISSING__,None,NaN,"{'value': 179.0414276123047, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-03
3,NaN,__MISSING__,None,NaN,"{'value': 171.3340606689453, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-04
4,NaN,__MISSING__,None,NaN,"{'value': 173.92208862304688, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-05
5,NaN,__MISSING__,None,NaN,"{'value': 167.70716857910156, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-06
6,NaN,__MISSING__,None,NaN,"{'value': 165.75071716308594, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-07
7,NaN,__MISSING__,None,NaN,"{'value': 241.08514404296875, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-08
8,NaN,__MISSING__,None,NaN,"{'value': 268.7997131347656, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-09
9,NaN,__MISSING__,None,NaN,"{'value': 166.55462646484375, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-10


In [67]:
preds['predicted_num_trips'].iloc[0]

{'value': 207.7970428466797,
 'tft_feature_importance': {'context_weights': array([0.0156209 , 0.12721986, 0.42763159, 0.15146005, 0.13639586,
         0.14167179]),
  'context_columns': array(['avg_tripduration', 'capacity', 'num_trips', 'pct_subscriber',
         'ratio_gender', 'starttime'], dtype=object),
  'horizon_weights': array([0.32863981, 0.67136014]),
  'horizon_columns': array(['capacity', 'starttime'], dtype=object),
  'attribute_weights': array([], dtype=float64),
  'attribute_columns': array([], dtype=object)}}

#### Batch Prediction Job - With Explainations

Request the batch prediction directly using the `batch_prediction()` method for the model - [SDK Reference](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_batch_predict).

In [68]:
batchjobExplain = forecast_model.batch_predict(
    job_display_name = f'{SERIES}_{EXPERIMENT}_predictions',
    bigquery_source = f"bq://{BQ_PROJECT}.{BQ_DATASET}.{EXPERIMENT}_horizon_input",
    instances_format = 'bigquery',
    bigquery_destination_prefix = f"bq://{BQ_PROJECT}.{BQ_DATASET}",
    predictions_format = 'bigquery',
    generate_explanation=True,
    sync = True
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/1026793852137/locations/us-central1/batchPredictionJobs/5418320665193218048
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/1026793852137/locations/us-central1/batchPredictionJobs/5418320665193218048')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5418320665193218048?project=1026793852137
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/5418320665193218048 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/5418320665193218048 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredictionJobs/5418320665193218048 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1026793852137/locations/us-central1/batchPredicti

In [69]:
batchjobExplain.output_info.bigquery_output_table

'predictions_2024_03_22T05_26_00_741Z_378'

In [70]:
predsExplain = bq.query(f"SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{batchjobExplain.output_info.bigquery_output_table}`  WHERE start_station_name = 'Central Park West & W 68 St' ORDER BY starttime, predicted_on_starttime").to_dataframe()
predsExplain

,avg_tripduration,capacity,explanation,num_trips,pct_subscriber,predicted_num_trips,predicted_on_starttime,ratio_gender,start_station_name,starttime
0,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 207.7970428466797, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-01
1,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 226.3101043701172, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-02
2,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 179.0414276123047, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-03
3,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 171.3340606689453, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-04
4,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 173.92208862304688, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-05
5,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 167.70716857910156, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-06
6,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 165.75071716308594, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-07
7,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 241.08514404296875, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-08
8,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 268.7997131347656, 'tft_feature_impo...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-09
9,NaN,__MISSING__,{'attributions': [{'featureAttributions': {'st...,None,NaN,"{'value': 166.55462646484375, 'tft_feature_imp...",2016-10-01,NaN,Central Park West & W 68 St,2016-10-10


In [71]:
predsExplain['predicted_num_trips'].iloc[0]

{'value': 207.7970428466797,
 'tft_feature_importance': {'context_weights': array([0.0156209 , 0.12721986, 0.42763159, 0.15146005, 0.13639586,
         0.14167179]),
  'context_columns': array(['avg_tripduration', 'capacity', 'num_trips', 'pct_subscriber',
         'ratio_gender', 'starttime'], dtype=object),
  'horizon_weights': array([0.32863981, 0.67136014]),
  'horizon_columns': array(['capacity', 'starttime'], dtype=object),
  'attribute_weights': array([], dtype=float64),
  'attribute_columns': array([], dtype=object)}}